# Result Analysis

This notebook calculate some structural feature of predicted AlphaFold structures.

In [2]:
# Requirements and dependencies, defined some functions for PyMOL analysis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json
from pymol import cmd, CmdException
import os,time


# get distance object information in PyMOL
def get_raw_distances(names='', state=1, selection='all', quiet=1):
    from chempy import cpv

    state, quiet = int(state), int(quiet)
    if state < 1:
        state = cmd.get_state()

    valid_names = cmd.get_names_of_type('object:measurement')
    if names == '':
        names = ' '.join(valid_names)
    else:
        for name in names.split():
            if name not in valid_names:
                print(' Error: no such distance object: ' + name)
                raise CmdException

    raw_objects = cmd.get_session(names, 1, 1, 0, 0)['names']

    xyz2idx = {}
    cmd.iterate_state(state, selection, 'xyz2idx[x,y,z] = (model,index)',space=locals())

    r = []
    for obj in raw_objects:
        try:
            points = obj[5][2][state - 1][1]
            if points is None:
                raise ValueError
        except (KeyError, ValueError, IndexError):
            continue
        for i in range(0, len(points), 6):
            xyz1 = tuple(points[i:i + 3])
            xyz2 = tuple(points[i + 3:i + 6])
            try:
                r.append((xyz2idx[xyz1], xyz2idx[xyz2], cpv.distance(xyz1, xyz2)))
                if not quiet:
                    print(' get_raw_distances: ' + str(r[-1]))
            except KeyError:
                if quiet < 0:
                    print(' Debug: no index for %s %s' % (xyz1, xyz2))
    return r

In [5]:
# create dataframe object for A501 proteome
df_A501 = pd.DataFrame(columns=["ID","name","sequence"])

# if you used the full proteome, you can use the following code to load the dataframe
# df_A501["ID"] = ["A501_%d"%(i+1) for i in range(302+1878)]   # uncomment this line to use the full proteome
# here we use the short proteome, you can use the following code to load the dataframe
df_A501["ID"] = ["A501_%d"%(i+1) for i in range(10)]
df_A501.head()

,ID,name,sequence
0,A501_1,NaN,NaN
1,A501_2,NaN,NaN
2,A501_3,NaN,NaN
3,A501_4,NaN,NaN
4,A501_5,NaN,NaN


In [4]:
structure_avail_list = []
sequence_list = []
sequence_len_list = []
for filename in df_A501["ID"]:
    if os.path.exists("A501_structure/%s/unrelaxed_model_1.pdb"%filename):
        structure_avail_list.append(True)
        cmd.load("A501_structure/%s/unrelaxed_model_1.pdb"%filename,"A501")  # load the structure
        sequence = ''.join(cmd.get_fastastr("all").split('\n')[1:])     # get sequence
        sequence_list.append(sequence)
        sequence_len_list.append(len(sequence))
        cmd.reinitialize()
    else:
        structure_avail_list.append(False)
        sequence_list.append(None)
        sequence_len_list.append(None)

 PyMOL not running, entering library mode (experimental)


In [6]:
# add the sequence and structure availability to the dataframe
df_A501["sequence"] = sequence_list
df_A501["sequence length"] = sequence_len_list
df_A501["structure avail"] = structure_avail_list
df_A501.head()

,ID,name,sequence,sequence length,structure avail
0,A501_1,NaN,MAKKGAGATRGISPVRPTRALPIGAYLKVADNSGAKVIQIIGVVGY...,141,True
1,A501_2,NaN,MGKSLIQQRRGKGTTTFRAPSHRYRGAVKYVPLNVVKEKTLRGVVE...,239,True
2,A501_3,NaN,MKVVRFGVSVPEELLEKFDRIIEEKGYVNRSEAIRDLMRDFIIRHE...,138,True
3,A501_4,NaN,MDCTKDYCVKDLSLATSGEKKIDWVSRFMPVLQTIRREFEREKPFK...,421,True
4,A501_5,NaN,MRRDYTLYLFASLGSFLIAYIALPLVIIFAKQLMDWEMLVKTLHDP...,247,True


In [7]:
# Calculate plddt
template_list_plddt = []
for index in df_A501["ID"]:
    if os.path.exists("A501_structure/%s/ranking_debug.json"%index):
        f_json = open("A501_structure/%s/ranking_debug.json"%index,"r")
        json_dict = json.load(f_json)
        try:
            plddt_1 = json_dict["plddts"]["model_1"]
            plddt_2 = json_dict["plddts"]["model_2"]
        except:
            plddt_1 = json_dict["plddts"]["model_1_ptm"]
            plddt_2 = json_dict["plddts"]["model_2_ptm"]
        template_list_plddt.append([index,plddt_1,plddt_2])
    else:
        template_list_plddt.append([index,None,None])
template_df_plddt = pd.DataFrame(template_list_plddt,columns=["ID","model_1_plddt","model_2_plddt"])
df_A501 = df_A501.merge(template_df_plddt,how="left",on="ID")
df_A501.head()

,ID,name,sequence,sequence length,structure avail,model_1_plddt,model_2_plddt
0,A501_1,NaN,MAKKGAGATRGISPVRPTRALPIGAYLKVADNSGAKVIQIIGVVGY...,141,True,90.681015,90.474789
1,A501_2,NaN,MGKSLIQQRRGKGTTTFRAPSHRYRGAVKYVPLNVVKEKTLRGVVE...,239,True,94.120890,93.913161
2,A501_3,NaN,MKVVRFGVSVPEELLEKFDRIIEEKGYVNRSEAIRDLMRDFIIRHE...,138,True,95.654313,95.708373
3,A501_4,NaN,MDCTKDYCVKDLSLATSGEKKIDWVSRFMPVLQTIRREFEREKPFK...,421,True,96.819289,96.925328
4,A501_5,NaN,MRRDYTLYLFASLGSFLIAYIALPLVIIFAKQLMDWEMLVKTLHDP...,247,True,95.500632,95.463742


In [8]:
# Calculate protein properties
disul_list = []
hbond_list = []
sbrge_list = []
surf__list = []
sasa__list = []
sstrc_list = []
for i in range(df_A501.shape[0]):
    if os.path.exists("A501_structure/%s/unrelaxed_model_1.pdb"%df_A501["ID"][i]):
        cmd.load("A501_structure/%s/unrelaxed_model_1.pdb"%df_A501["ID"][i],"%s_1"%df_A501["ID"][i])   # load pdb file

        cmd.distance("disul","name SG","name SG",cutoff=2.1)     # disulfide bond
        cmd.distance("hbond","all","all",mode=2)                 # hydrogen bond
        cmd.distance("saltbridge","(resn ASP and name OD2) or (resn GLU and name OD2)","(resn ARG and (name NH1 or name NH2)) or (resn LYS and name NZ) or (resn HIS and name NE2)",cutoff=4)  # salt bridge
        cmd.dss()
        ss_string = ""
        for a in cmd.get_model(df_A501["ID"][i] +" and n. ca").atom:
            ss_string = ss_string+a.ss 


        disul_list.append(len(get_raw_distances("disul")))       # disulfide bond
        hbond_list.append(len(get_raw_distances("hbond")))       # hydrogen bond
        sbrge_list.append(len(get_raw_distances("saltbridge")))  # salt bridge
        surf__list.append(cmd.get_area('all'))                   # protein surface
        cmd.set('dot_solvent',1)
        sasa__list.append(cmd.get_area('all'))                   # SASA
        sstrc_list.append(ss_string)                             # secondary structure

        cmd.reinitialize()
    else:
        disul_list.append(None)       # disulfide bond
        hbond_list.append(None)       # hydrogen bond
        sbrge_list.append(None)  # salt bridge
        surf__list.append(None)                   # protein surface
        sasa__list.append(None)                   # SASA
        sstrc_list.append(None)                             # secondary structure

    print("\r%d/%d"%(i+1,df_A501.shape[0]),end="")


10/10

In [10]:
# Add calculated data to dataframe
df_A501["disulfide_bond"] = disul_list
df_A501["hydrogen_bond"] = hbond_list
df_A501["salt_bridge"] = sbrge_list
df_A501["surface"] = surf__list
df_A501["SASA"] = sasa__list
df_A501["secondary_structure"] = sstrc_list


df_A501.to_csv("A501_results.csv",index=False)  # save A501 result to csv file
df_A501.head()

,ID,name,sequence,sequence length,structure avail,model_1_plddt,model_2_plddt,disulfide_bond,hydrogen_bond,salt_bridge,surface,SASA,secondary_structure
0,A501_1,NaN,MAKKGAGATRGISPVRPTRALPIGAYLKVADNSGAKVIQIIGVVGY...,141,True,90.681015,90.474789,0,121,2,16448.005859,8913.559570,LLLLLLLLLLLLLLLLLLLLLLLSSSSSSLLLLLLSSSSSSSSLLL...
1,A501_2,NaN,MGKSLIQQRRGKGTTTFRAPSHRYRGAVKYVPLNVVKEKTLRGVVE...,239,True,94.120890,93.913161,0,183,2,28280.498047,16099.310547,LLLLLHHHHHHHLLHHHLLLHHHLLLLLLLLLHHHHLLLLSSSSSS...
2,A501_3,NaN,MKVVRFGVSVPEELLEKFDRIIEEKGYVNRSEAIRDLMRDFIIRHE...,138,True,95.654313,95.708373,0,146,5,16841.468750,10151.217773,LLLLLLLLLLLHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHH...
3,A501_4,NaN,MDCTKDYCVKDLSLATSGEKKIDWVSRFMPVLQTIRREFEREKPFK...,421,True,96.819289,96.925328,1,523,19,49283.140625,19119.728516,LSSSLLSSSLLHHHHHHHHHHHHHHHHHLHHHHHHHHHHHHHLLLL...
4,A501_5,NaN,MRRDYTLYLFASLGSFLIAYIALPLVIIFAKQLMDWEMLVKTLHDP...,247,True,95.500632,95.463742,0,316,3,28950.642578,15324.120117,LLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLHHHHHHHHHLH...


In [11]:
# start a similar process for 3DAC proteome
df_3DAC = pd.DataFrame(columns=["ID","name","sequence"])

# if you used the full proteome, you can use the following code to load the dataframe
# df_3DAC["ID"] = ["3DAC_%d"%(i+1) for i in range(302+1196)]   # uncomment this line to use the full proteome
# here we use the short proteome, you can use the following code to load the dataframe
df_3DAC["ID"] = ["3DAC_%d"%(i+1) for i in range(10)]
df_3DAC.head()

,ID,name,sequence
0,3DAC_1,NaN,NaN
1,3DAC_2,NaN,NaN
2,3DAC_3,NaN,NaN
3,3DAC_4,NaN,NaN
4,3DAC_5,NaN,NaN


In [12]:
structure_avail_list = []
sequence_list = []
sequence_len_list = []
for filename in df_3DAC["ID"]:
    try:
        os.rename("3DAC_structure/%s/unrelaxed_model_1_ptm.pdb"%filename,"3DAC_structure/%s/unrelaxed_model_1.pdb"%filename)
        os.rename("3DAC_structure/%s/unrelaxed_model_2_ptm.pdb"%filename,"3DAC_structure/%s/unrelaxed_model_2.pdb"%filename)
    except:
        pass
    if os.path.exists("3DAC_structure/%s/unrelaxed_model_1.pdb"%filename):
        structure_avail_list.append(True)
        cmd.load("3DAC_structure/%s/unrelaxed_model_1.pdb"%filename,"3DAC")
        sequence = ''.join(cmd.get_fastastr("all").split('\n')[1:])
        sequence_list.append(sequence)
        sequence_len_list.append(len(sequence))
        cmd.reinitialize()
    else:
        structure_avail_list.append(False)
        sequence_list.append(None)
        sequence_len_list.append(None)
df_3DAC["sequence"] = sequence_list
df_3DAC["sequence length"] = sequence_len_list
df_3DAC["structure avail"] = structure_avail_list

df_3DAC.head()

,ID,name,sequence,sequence length,structure avail
0,3DAC_1,NaN,MAQKKGSSQKLLVWILVGFALGIVGGLILGKDNVIWVAWMGDVFIR...,409,True
1,3DAC_2,NaN,MSTMPSKVVGIIGGMGPEAGVDLVYKVIKLSPAKRDQEHIHVILDN...,232,True
2,3DAC_3,NaN,MAQVKEVRLYTYQWKAEPPIGVSVLIIHGLGEHAGRYKSLVKALND...,264,True
3,3DAC_4,NaN,MYSQFKKLVKDAIRKADIEKRLDEIITYRRELHQIPEVGLELPKTK...,394,True
4,3DAC_5,NaN,MNEMFVVQAKFIGKESQKKLEGKTVAIVGCGGLGSPLVQMAARSGI...,230,True


In [13]:
# Calculate plddt
template_list_plddt = []
for index in df_3DAC["ID"]:
    if os.path.exists("3DAC_structure/%s/ranking_debug.json"%index):
        f_json = open("3DAC_structure/%s/ranking_debug.json"%index,"r")
        json_dict = json.load(f_json)
        try:
            plddt_1 = json_dict["plddts"]["model_1"]
            plddt_2 = json_dict["plddts"]["model_2"]
        except:
            plddt_1 = json_dict["plddts"]["model_1_ptm"]
            plddt_2 = json_dict["plddts"]["model_2_ptm"]
        template_list_plddt.append([index,plddt_1,plddt_2])
    else:
        template_list_plddt.append([index,None,None])
template_df_plddt = pd.DataFrame(template_list_plddt,columns=["ID","model_1_plddt","model_2_plddt"])
df_3DAC = df_3DAC.merge(template_df_plddt,how="left",on="ID")
df_3DAC.head()

,ID,name,sequence,sequence length,structure avail,model_1_plddt,model_2_plddt
0,3DAC_1,NaN,MAQKKGSSQKLLVWILVGFALGIVGGLILGKDNVIWVAWMGDVFIR...,409,True,85.773312,88.517354
1,3DAC_2,NaN,MSTMPSKVVGIIGGMGPEAGVDLVYKVIKLSPAKRDQEHIHVILDN...,232,True,96.532346,96.244699
2,3DAC_3,NaN,MAQVKEVRLYTYQWKAEPPIGVSVLIIHGLGEHAGRYKSLVKALND...,264,True,92.729368,92.923812
3,3DAC_4,NaN,MYSQFKKLVKDAIRKADIEKRLDEIITYRRELHQIPEVGLELPKTK...,394,True,94.767609,95.085076
4,3DAC_5,NaN,MNEMFVVQAKFIGKESQKKLEGKTVAIVGCGGLGSPLVQMAARSGI...,230,True,94.804814,95.198690


In [14]:
# Calculate protein properties
disul_list = []
hbond_list = []
sbrge_list = []
surf__list = []
sasa__list = []
sstrc_list = []
for i in range(df_3DAC.shape[0]):
    if os.path.exists("3DAC_structure/%s/unrelaxed_model_1.pdb"%df_3DAC["ID"][i]):
        cmd.load("3DAC_structure/%s/unrelaxed_model_1.pdb"%df_3DAC["ID"][i],"%s_1"%df_3DAC["ID"][i])   # load pdb file

        cmd.distance("disul","name SG","name SG",cutoff=2.1)     # disulfide bond
        cmd.distance("hbond","all","all",mode=2)                 # hydrogen bond
        cmd.distance("saltbridge","(resn ASP and name OD2) or (resn GLU and name OD2)","(resn ARG and (name NH1 or name NH2)) or (resn LYS and name NZ) or (resn HIS and name NE2)",cutoff=4)  # salt bridge
        cmd.dss()
        ss_string = ""
        for a in cmd.get_model(df_3DAC["ID"][i] +" and n. ca").atom:
            ss_string = ss_string+a.ss 



        disul_list.append(len(get_raw_distances("disul")))       # disulfide bond
        hbond_list.append(len(get_raw_distances("hbond")))       # hydrogen bond
        sbrge_list.append(len(get_raw_distances("saltbridge")))  # salt bridge
        surf__list.append(cmd.get_area('all'))                   # protein surface
        cmd.set('dot_solvent',1)
        sasa__list.append(cmd.get_area('all'))                   # SASA
        sstrc_list.append(ss_string)                             # secondary structure

        cmd.reinitialize()
    else:
        disul_list.append(None)       # disulfide bond
        hbond_list.append(None)       # hydrogen bond
        sbrge_list.append(None)  # salt bridge
        surf__list.append(None)                   # protein surface
        sasa__list.append(None)                   # SASA
        sstrc_list.append(None)                             # secondary structure

    print("\r%d/%d"%(i+1,df_3DAC.shape[0]),end="")


10/10

In [15]:
# Add calculated data to dataframe
df_3DAC["disulfide_bond"] = disul_list
df_3DAC["hydrogen_bond"] = hbond_list
df_3DAC["salt_bridge"] = sbrge_list
df_3DAC["surface"] = surf__list
df_3DAC["SASA"] = sasa__list
df_3DAC["secondary_structure"] = sstrc_list

df_3DAC.to_csv("3DAC_results.csv",index=False)
df_3DAC.head()

,ID,name,sequence,sequence length,structure avail,model_1_plddt,model_2_plddt,disulfide_bond,hydrogen_bond,salt_bridge,surface,SASA,secondary_structure
0,3DAC_1,NaN,MAQKKGSSQKLLVWILVGFALGIVGGLILGKDNVIWVAWMGDVFIR...,409,True,85.773312,88.517354,0,522,2,46165.542969,18625.396484,LLLLLHHHHHHHHHHHHHHHHHHHHHHHHLHHHHHHHHHHHHHHHH...
1,3DAC_2,NaN,MSTMPSKVVGIIGGMGPEAGVDLVYKVIKLSPAKRDQEHIHVILDN...,232,True,96.532346,96.244699,0,249,5,26818.630859,10773.984375,LLLLLLLLSSSSLLLLHHHHHHHHHHHHHHLLLLLHHHLLLSSSSL...
2,3DAC_3,NaN,MAQVKEVRLYTYQWKAEPPIGVSVLIIHGLGEHAGRYKSLVKALND...,264,True,92.729368,92.923812,0,304,12,30916.574219,12734.734375,LLLLLLLLLLSSSSLLLLLLLSSSSSSSLLLLLHHHHHHHHHHHHH...
3,3DAC_4,NaN,MYSQFKKLVKDAIRKADIEKRLDEIITYRRELHQIPEVGLELPKTK...,394,True,94.767609,95.085076,0,469,10,45688.320312,18097.576172,LHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLLLLLHHHH...
4,3DAC_5,NaN,MNEMFVVQAKFIGKESQKKLEGKTVAIVGCGGLGSPLVQMAARSGI...,230,True,94.804814,95.198690,0,258,5,26502.291016,10770.962891,LLHHHHHHHHHHLHHHHHHHHHSSSSSSLLLLHHHHHHHHHHHHLL...
